In [1]:
import pandas as pd
from datetime import datetime

In [2]:
import dask.dataframe as dd

In [3]:
with open("wowah_data_out.csv", "r", encoding = "utf-8") as f:
    count = 1
    for line in f:
        print(line)
        count += 1
        if count > 10:
            break

timestamp,char,guild,level,race,class,region,file

12/31/05 23:59:46,0,-1,5,Orc,Warrior,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:46,1,-1,9,Orc,Shaman,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:52,2,-1,13,Orc,Shaman,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:52,3,0,14,Orc,Warrior,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:52,4,-1,14,Orc,Shaman,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:52,5,-1,16,Orc,Hunter,The Barrens,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:52,6,-1,18,Orc,Warlock,The Barrens,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:52,7,-1,17,Orc,Hunter,Silverpine Forest,import/2006_01_03/2006-01-01/00-03-56.txt

12/31/05 23:59:57,8,0,26,Orc,Warrior,Stonetalon Mountains,import/2006_01_03/2006-01-01/00-03-56.txt



In [4]:
imported_date_parser = lambda x: datetime.strptime(x, "%m/%d/%y %H:%M:%S")

In [5]:
imported_date_parser("12/31/05 23:59:46")

datetime.datetime(2005, 12, 31, 23, 59, 46)

In [6]:
df = dd.read_csv("wowah_data_out.csv", parse_dates = ["timestamp"], date_parser = imported_date_parser)

In [7]:
df.head()

,timestamp,char,guild,level,race,class,region,file
0,2005-12-31 23:59:46,0,-1,5,Orc,Warrior,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt
1,2005-12-31 23:59:46,1,-1,9,Orc,Shaman,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt
2,2005-12-31 23:59:52,2,-1,13,Orc,Shaman,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt
3,2005-12-31 23:59:52,3,0,14,Orc,Warrior,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt
4,2005-12-31 23:59:52,4,-1,14,Orc,Shaman,Durotar,import/2006_01_03/2006-01-01/00-03-56.txt


In [8]:
df.dtypes

timestamp    datetime64[ns]
char                  int64
guild                 int64
level                 int64
race                 object
class                object
region               object
file                 object
dtype: object

In [10]:
races = df["race"].value_counts().compute(scheduler='processes', num_workers=4)

In [11]:
races

Undead       11365578
Tauren        7985224
Blood Elf     7756519
Troll         5464194
Orc           3901512
547人            29255
373族            11353
3033                6
74622妖              3
27410               3
Name: race, dtype: int64

In [13]:
classes = df["class"].value_counts().compute(scheduler='threads', num_workers=4)

KeyboardInterrupt: 

In [ ]:
df["class"].value_counts()

In [ ]:
df["region"].value_counts()

In [ ]:
df[df["race"] == "547人"]["class"].unique()

In [14]:
def is_ascii(s):
    try:
        s.encode('ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

In [15]:
is_ascii("547")

True

In [17]:
df["race"].apply(is_ascii)

C:\Users\ryanc\.conda\envs\PythonData\lib\site-packages\dask\dataframe\core.py:3150: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('race', 'bool'))

  warnings.warn(meta_warning(meta))


Dask Series Structure:
npartitions=58
    bool
     ...
    ... 
     ...
     ...
Name: race, dtype: bool
Dask Name: apply, 290 tasks

In [18]:
cleaned_df = df[df["race"].apply(is_ascii).compute(scheduler='processes', num_workers=4)]
len(cleaned_df)

C:\Users\ryanc\.conda\envs\PythonData\lib\site-packages\dask\dataframe\core.py:3465: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  meta = self._meta[_extract_meta(key)]
C:\Users\ryanc\.conda\envs\PythonData\lib\site-packages\dask\core.py:121: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return func(*(_execute_task(a, cache) for a in args))


KeyboardInterrupt: 

In [ ]:
cleaned_df = cleaned_df[cleaned_df["class"].apply(is_ascii)]
len(cleaned_df)

In [ ]:
cleaned_df = cleaned_df[pd.isnull(pd.to_numeric(cleaned_df["region"], errors="coerce"))]
len(cleaned_df)

In [ ]:
len(df) - len(cleaned_df)

In [ ]:
cleaned_df.head()

In [ ]:
trace_df = cleaned_df["file"].astype(str).str.split("/").apply(pd.Series)

In [ ]:
trace_df.head()